In [ ]:
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

In [ ]:
from learners import *
from dataset import *
from explanations import *

## Load Dataset

In [ ]:
credit = pd.read_csv("taiwan_credit.csv")
X, y = load_credit()

## Train Model

In [ ]:
w_star = train_logistic_regression(X, y)
'''
Need to add offset before evaluate
'''
X1 = append_offset(X)
evaluate_linear(w_star, X1, y)

In [ ]:
'''
Compute labels and all margin distances
'''
y = label_linear_dataset(w_star, X1)

In [ ]:
params = {"R": 0.2, "thres": 0.1, "start_pct": 0,
          "per_cluster": 50, "num_ws": 200, 
          "mix_time": 200, "r": 0.3, "range": 15,
           "pool_size": 8}

In [ ]:
'''
Reduce the per_cluster to make number of explanations smaller for full
50 per cluster -> 600
'''
exp_X, exp_Y = kmedoid_summary(X1, y, per_cluster=params["per_cluster"])

In [ ]:
'''
MMD Critic

exp_X, exp_Y = run_mmd(X1, y, num_prototype=550, num_crit=50, gamma=0.1)
'''

## Sample hyperplanes

In [ ]:
pcts = []

In [ ]:
'''
Just sample the ws and then analyze them
'''
from multiprocessing import Pool
PREFIX = "linear_exp_percentile"

In [ ]:
for i in range(1, 18):
    pct = params["start_pct"] + 5 * i
    pcts.append(pct)
    
    allowed_X, allowed_y = linear_explanation_policy(exp_X, exp_Y, w_star, cutoff_percentile=pct)
    
    rev_sign_X = apply_sign(allowed_X, allowed_y)
    w_star = w_star.reshape(-1,1)
    
    def polytope_sample(x): 
        return random_walk_polytope(rev_sign_X, w_star, n_samples=x, mix_time=params["mix_time"], r=params["r"])

    sampled_ws = None
    with Pool(params["pool_size"]) as pool:
        sampled_ws = pool.map(polytope_sample, [10 for i in range(params["pool_size"])])
    sampled_ws = np.vstack(sampled_ws)
    np.save(PREFIX + "ws_1_pct_{}.npy".format(pct), sampled_ws)

    sampled_ws2 = None
    with Pool(params["pool_size"]) as pool:
        sampled_ws2 = pool.map(polytope_sample, [10 for i in range(params["pool_size"])])
    sampled_ws2 = np.vstack(sampled_ws2)
    np.save(PREFIX + "ws_2_pct_{}.npy".format(pct), sampled_ws2)
    
    sampled_ws3 = None
    with Pool(params["pool_size"]) as pool:
        sampled_ws3 = pool.map(polytope_sample, [10 for i in range(params["pool_size"])])
    sampled_ws3 = np.vstack(sampled_ws3)
    np.save(PREFIX + "ws_3_pct_{}.npy".format(pct), sampled_ws3)
    
    sampled_ws4 = None
    with Pool(params["pool_size"]) as pool:
        sampled_ws4 = pool.map(polytope_sample, [10 for i in range(params["pool_size"])])
    sampled_ws4 = np.vstack(sampled_ws4)
    np.save(PREFIX + "ws_4_pct_{}.npy".format(pct), sampled_ws4)
            
    sampled_ws5 = None
    with Pool(params["pool_size"]) as pool:
        sampled_ws5 = pool.map(polytope_sample, [10 for i in range(params["pool_size"])])
    sampled_ws5 = np.vstack(sampled_ws5)
    np.save(PREFIX + "ws_5_pct_{}.npy".format(pct), sampled_ws5)